In [20]:
%matplotlib inline
import os
import sys
import pandas as pd

In [34]:
# %load "../src/features/build_features.py"
from operator import xor
import pandas as pd
import networkx as nx
import torch
import numpy as np
import os
from sklearn.preprocessing import StandardScaler

import sys
sys.path.insert(0, 'src/model')
from utils import encode_onehot, frac_mat_power

def load_data(path, dataset, train, val, test):
    """Load network dataset"""
    print('Loading {} dataset...'.format(dataset))
    
    # Load data
    nodes = pd.read_csv('{}{}target.csv'.format(path, dataset), sep=',')
    edges = pd.read_csv('{}{}edges.csv'.format(path, dataset), sep=',')
    
    # Construct graph
    G = nx.Graph(name = 'G')

    # Create nodes
    for i in nodes.iloc[:, -1]:
        G.add_node(i, name=i)

    # Create edges
    for e in edges.to_numpy():
        G.add_edge(e[0], e[1])

    #See graph info
    print('Graph Info:\n', nx.info(G))
    
    #Get the Adjacency Matrix (A) and Node Features Matrix (X) as numpy array
    A = torch.FloatTensor(nx.adjacency_matrix(G).todense())
    X = nodes.iloc[:, [1, 3, 4]].to_numpy().astype(float)
    y = encode_onehot(nodes.iloc[:, 2].to_numpy())

    idx_train = range(train)
    idx_val = range(train, train+val)
    idx_test = range(train+val, train+val+test)

    I = torch.eye(A.shape[0]) #create Identity Matrix of A
    A_hat = A + I #add self-loop to A
    D = torch.diag(torch.sum(A_hat, axis=0)) #create Degree Matrix of A
    D_half_norm = frac_mat_power(D, -0.5) #calculate D to the power of -0.5
    A = D_half_norm.mm(A_hat).mm(D_half_norm)

    # Standardize X
    scaler = StandardScaler()
    X = scaler.fit_transform(X)
    X = torch.FloatTensor(X)

    y = torch.LongTensor(np.where(y)[1])

    idx_train = torch.LongTensor(idx_train)
    idx_val = torch.LongTensor(idx_val)
    idx_test = torch.LongTensor(idx_test)

    print('Shape of A: ', A.shape)
    print('\nShape of X: ', X.shape)
    print('\nAdjacency Matrix (A):\n', A)
    print('\nNode Features Matrix (X):\n', X)
    
    return A, X, y, idx_train, idx_val, idx_test

In [36]:
A, X, y, idx_train, idx_val, idx_test = load_data("../src/data/twitch/", "musae_ENGB_", 1000, 2000, 3000)

Loading musae_ENGB_ dataset...
Graph Info:
 Name: G
Type: Graph
Number of nodes: 7126
Number of edges: 35324
Average degree:   9.9141


/home/s3patil/DSC 180/DSC180-Replication-Project/notebooks/../src/model/utils.py:13: UserWarning: torch.eig is deprecated in favor of torch.linalg.eig and will be removed in a future PyTorch release.
torch.linalg.eig returns complex tensors of dtype cfloat or cdouble rather than real tensors mimicking complex tensors.
L, _ = torch.eig(A)
should be replaced with
L_complex = torch.linalg.eigvals(A)
and
L, V = torch.eig(A, eigenvectors=True)
should be replaced with
L_complex, V_complex = torch.linalg.eig(A) (Triggered internally at  /pytorch/aten/src/ATen/native/BatchLinearAlgebra.cpp:2897.)
  evals, evecs = torch.eig (m, eigenvectors = True)  # get eigendecomposition


Shape of A:  torch.Size([7126, 7126])

Shape of X:  torch.Size([7126, 3])

Adjacency Matrix (A):
 tensor([[0.1250, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0500, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.1000,  ..., 0.0000, 0.0000, 0.0000],
        ...,
        [0.0000, 0.0000, 0.0000,  ..., 0.2500, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0476, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.3333]])

Node Features Matrix (X):
 tensor([[-0.0937, -0.0620, -0.2387],
        [ 0.1469, -0.0640, -0.2387],
        [-1.5768, -0.0495, -0.2387],
        ...,
        [-0.1517, -0.0639, -0.2387],
        [ 0.7370, -0.0631, -0.2387],
        [ 0.3846, -0.0640, -0.2387]])
